In [756]:
import pandas as pd
import numpy as np

### Layout 1

In [757]:
df= pd.read_excel('location.xlsx', sheet_name= 'Layout #1') #Read in the macro layouts

In [758]:
df1= pd.read_excel('location.xlsx', sheetname= 'Layout #1', header=None) #Read in the macro layouts a different way.
#Delete underscore
df1 = df1.drop([0],axis = 0)
df12 = df1.replace(np.nan, '', regex=True)

df12[1][1] # First element begins at [1][1]. This is important


'A-13-S61-P1'

In [759]:
data= pd.read_excel('data.xlsx', sheet_name= 'Sheet1',header=None) #Read in the broken panel locations

In [760]:
#Only hit this block once
A = data.iloc[:,0] # Select the location column in data
Macro_Label = [] #initialize the macrolocation list (2A-4, 3B-7, etc)
for i,element in enumerate(A): # Loop through the column
    Macro_Label.append(element[0:4]) #Append the macro labels
    data.iloc[i,0] = element[4:len(element)] #Delete the macro labels from data
    
data.head()

,0,1
0,A-11-S21-P14,120919-31-M285MI-0038
1,A-11-S31-P2,120914-31-M285MI-0089
2,A-11-S32-P21,120913-31-M285MI-0812
3,A-11-S33-P53,120913-31-M285MI-1059
4,A-11-S53-P44,120913-31-M285MI-1091


In [761]:
locations= data[data.columns[0]].values #Split up locations into its own column
serials= data[data.columns[1]].values #Split up serials into its own column
Sorting_dict = {} #Initialize a dictionary that sorts everything by macrolocation
for i, element in enumerate(Macro_Label): #This loop stores all similar locations under the same macrolable. 
    if element in Sorting_dict: # The dictionary is read Loc1, serial 1, Loc2, Serial 2, .....
        Sorting_dict[element].append(locations[i])
        Sorting_dict[element].append(serials[i])
    else: 
        Sorting_dict[element] = [locations[i]]
        Sorting_dict[element].append(serials[i])

In [762]:
#locations_big= np.random.choice(locations,20000)
#serials_big= np.random.choice(serials,20000)
#print(df12[0][1], df12[5][2], df12[3][2])

In [763]:
def fragment(df): #This function takes in a large data frames (df) and splits it up into four smaller ones based on inverter labels
    letters=['A','B','C','D'] #Inverter labels
    n= 0 #Used as a counter
    i = 1 #We are always drawing from the second column of the dataframe. Starting from the second generates an error.
    DF_list = [[],[],[],[]] #Initilize the list of smaller data frame outputs
    for r,j in enumerate(letters): #This loop creates the smaller dataframes
        P = 0.1 #These values have meaning in the four loop
        P1 = 0.1
        for f,k in enumerate(df.T): #We take the transpose here so that f numbers from 0 to however many rows there are
            n = f+1 #Because the first element starts at [1][1]
            if df[i][n] == '': # Skip the blank space but don't change the P values
                continue
            elif j in df[i][n]: 
                if P == .1:# Record the first location that shows any given inverter
                    P = n
                    #print(P)
            elif P != .1: # Record the last location that shows any given inverter
                P1 = n
                #print(P1)
                break
            else:
                continue
        if P1 == .1: #Generate the list of lists from the P values
            DF_list[r] = df.iloc[P-1:,:] #The last inverter needs to cover the rest of the corresponding list
        else:
            DF_list[r] = df.iloc[P-1:P1-1,:]
        
    return DF_list #List of dataframes

In [764]:
o = fragment(df12) #Call the fragment function
L = len(DF_list) 
for i in range(0,L): #Loop through all elements of the list of dataframes
    temp=DF_list[i].iloc[0,:] #Generate a temp dataframe
    labels=[]
    for y in temp:
        if len(y) != 0: 
            labels.append(y[0:7]) #Label all columns with the appropriate headers (ex. A-11-S5)
        else:
            labels.append(y)
    DF_list[i].columns=labels
    DF_list[i]['buffer'] = '' #Buffer the dataframe with an empty column so serial numbers can be placed in this column
DF_list[1].head()

,,B-13-S6,,B-13-S5,,B-13-S4,,B-13-S3,,B-13-S2,...,,,,,,,,,,buffer
64,,B-13-S61-P1,,B-13-S51-P1,,B-13-S41-P1,,B-13-S31-P1,,B-13-S21-P1,...,,,,,,,,,,
65,,B-13-S61-P2,,B-13-S51-P2,,B-13-S41-P2,,B-13-S31-P2,,B-13-S21-P2,...,,,,,,,,,,
66,,B-13-S61-P3,,B-13-S51-P3,,B-13-S41-P3,,B-13-S31-P3,,B-13-S21-P3,...,,,,,,,,,,
67,,B-13-S61-P4,,B-13-S51-P4,,B-13-S41-P4,,B-13-S31-P4,,B-13-S21-P4,...,,,,,,,,,,
68,,B-13-S61-P5,,B-13-S51-P5,,B-13-S41-P5,,B-13-S31-P5,,B-13-S21-P5,...,,,,,,,,,,


Sorting Algorithm. We will also need to sort data into proper layout. This should be done externally, as filtering within function will increase execution time.

'X Y'

In [823]:
def map(z): #Takes in the locations of all failed modules
    da=DF_list[0].copy(deep=True) #All dataframes are put into a dictionary (below)
    db=DF_list[1].copy(deep=True)
    dc=DF_list[2].copy(deep=True)
    dd=DF_list[3].copy(deep=True)
    temp=[da,db,dc,dd]
    dic={'A':da, 'B':db, 'C':dc, 'D':dd}
    for i,n in enumerate(z):
        inverter= n[0] #n[4] #Selects proper inverter
        in_map= dic[inverter] #Selects proper dataframe, given the inverter
        column = n[0:7] #Selects the proper column
        row = in_map[column][:].values.tolist().index(n) #Select the proper row.
        col= in_map.columns.get_loc(column)
        in_map.iloc[row,(col+1)]=serials[i] #Input the sserial numbers
        in_map.iloc[row,col] = 'X '+n #Renames the cell location for later excel formatting
    for i in temp:
        i.columns= np.arange(len(i.columns))
    return [da,db,dc,dd]

In [830]:
format1 = workbook.add_format({'font_color': '#9C0006'})

for item in Sorting_dict: #Loop through all macro locations and generate a map based on the results
    locations = Sorting_dict[item][0::2] #Locations are every other element
    serials = Sorting_dict[item][1::2] #Serials are every other element
    s= map(locations) #Call the mapping function
    final= pd.concat(s, axis=0, ignore_index=True)# sort=False)
   
    
    writer = pd.ExcelWriter('pandas_conditional.xlsx', engine='xlsxwriter')
    final.to_excel(writer, '%s.xlsx'%item) # Output results to an excel file
    workbook  = writer.book
    worksheet = writer.sheets['%s.xlsx'%item ]
    worksheet.conditional_format('A1', {'type':     'cell',
                                    'criteria': 'containing',
                                    'value':     'X ',
                                    'format':    format1})
    writer.save()